1. Load Dataset
2. Define Parametrized Model
3. Instantiate Model
4. Instantiate Optimizer
5. Training/Eval Loop
    1. Set Gradients to Zero
    2. Compute out Estimation (Forward Pass)
    3. Compute our Loss (Objective Function to Minimize)
    4. Compute out Gradients (Backward Pass)
    5. Update our Weights
    6. Monitoring (Visualize Training Loss / Eval Loss)
6. Save our Weights
7. Export our Model for WebGPU

In [1]:
from typing import Callable
from tinygrad import Tensor, TinyJit
from tinygrad.nn.datasets import mnist
from tinygrad.nn.optim import Adam
from tinygrad.helpers import trange
import tinygrad.nn as nn

In [ ]:
class Model:
    def __init__(self) : 
        self.layers: list[Callable[[Tensor], Tensor]] = [
            lambda x: x.flatten(1),
            nn.Linear(784, 512), Tensor.relu,
            nn.Linear(512, 512), Tensor.relu,
            nn.Linear(512, 10)
        ]

    def __call__(self, x: Tensor) -> Tensor:
        return x.sequential(self.layers)

In [3]:
X_train,Y_train, X_test, Y_test = mnist()
model = Model()
opt = Adam(nn.state.get_parameters(model))

In [4]:
@TinyJit
@Tensor.train()
def train_step() -> Tensor:
    opt.zero_grad()
    samples = Tensor.randint(512, high=int(X_train.shape[0]))
    loss = model(X_train[samples]).sparse_categorical_crossentropy(Y_train[samples]).backward()
    return loss.realize(*opt.schedule_step())

@TinyJit
def get_test_acc() -> Tensor:
    return model(X_test).argmax(axis=1) == (Y_test).mean() * 100

In [5]:
test_acc = float('nan')
for step in (t := trange(100)):
    loss = train_step().item
    if step % 10 == 9: test_acc = get_test_acc().item()
    t.set_description(f"loss: {loss:4.2f} test acc: {test_acc:4.2f}%")

  0%|                                                   | 0/100 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'clang'